# Chapter 4 — C3D Plotting: GRFs & Markers

> **This notebook belongs to Chapter 4 of the eBook.**  
> It may require libraries that you installed in previous chapters.  
> **Author:** Hossein Mokhtarzadeh — **Poseiq.com**  
> More of our work and courses are available on **Udemy**.

In this chapter we will:
- Load a `.c3d` file
- Explore parameters (including force plate metadata)
- Plot **all analog channels** (including GRFs if present)
- Plot **marker trajectories**
- Use a **generic C3D Parameter Visualizer** widget


## Setup

If you haven't already from earlier chapters, install/enable the following:

```bash
pip install ezc3d ipywidgets matplotlib pandas numpy
jupyter nbextension enable --py widgetsnbextension
```


In [ ]:
!pip install ezc3d ipywidgets matplotlib pandas numpy
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
# ===== Imports & basic setup =====
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display, HTML

# Main C3D reader
import ezc3d

# Optional widgets (used by the visualizer)
import ipywidgets as W

# Helper: nicer plots
plt.rcParams['figure.figsize'] = (7, 3.5)
plt.rcParams['axes.grid'] = True


## Load a C3D file

Update `c3d_path` to point to your file. We parse points (markers) and analog channels.


In [ ]:
# ===== Load C3D =====
# Change this path to your file (relative to this notebook or absolute)
c3d_path = Path('/content/Eb015pi.c3d') # or Path(./trial.c3d)

assert c3d_path.exists(), f"Couldn't find {c3d_path}. Update the path above."

c3d_obj = ezc3d.c3d(str(c3d_path))

# We'll expose a dict-like API similar to the one used elsewhere in the book
c3d = {
    'parameters': c3d_obj['parameters'],
    'data': {
        'points': c3d_obj['data']['points'],    # (4, nMarkers, nFrames)
        'analogs': c3d_obj['data']['analogs']   # (nSubframes, nAnalogs, nFrames)
    },
    'point_rate': float(c3d_obj['header']['points']['frame_rate']),
    'analog_rate': float(c3d_obj['header']['analogs']['frame_rate']),
    'info': c3d_obj,
}
print('Loaded:', c3d_path.name)
print('Markers:', c3d['data']['points'].shape, ' Analogs:', c3d['data']['analogs'].shape)
print('Rates  — points:', c3d['point_rate'], 'Hz | analog:', c3d['analog_rate'], 'Hz')


In [ ]:
# ===== Names for markers and analog channels =====
def get_marker_labels(c3d_obj):
    P = c3d_obj['parameters']
    try:
        labels = P['POINT']['LABELS']['value']
        return [str(x) for x in labels]
    except Exception:
        n = c3d_obj['data']['points'].shape[1]
        return [f'Marker_{i}' for i in range(n)]

def get_analog_labels(c3d_obj):
    P = c3d_obj['parameters']
    for key in ['ANALOG', 'ANALOGS']:
        if key in P and 'LABELS' in P[key]:
            return [str(x) for x in P[key]['LABELS']['value']]
    n = c3d_obj['data']['analogs'].shape[1]
    return [f'Analog_{i}' for i in range(n)]

marker_labels = get_marker_labels(c3d_obj)
analog_labels = get_analog_labels(c3d_obj)

print('First few markers:', marker_labels[:5])
print('First few analogs:', analog_labels[:5])


In [ ]:
# ===== Build time vectors =====
points = c3d['data']['points']
analogs = c3d['data']['analogs']

n_frames_points = points.shape[2]
n_frames_analogs = analogs.shape[2]
subframes = analogs.shape[0]

t_points = np.arange(n_frames_points) / c3d['point_rate']
t_analogs = np.arange(n_frames_analogs * subframes) / c3d['analog_rate']


## Plot all analog channels

This will iterate each analog channel and draw it as its own figure.


In [ ]:
# ===== Plot all analog channels =====
def plot_all_analogs(analogs, labels, t_analogs):
    subframes, nA, nF = analogs.shape
    data = analogs.transpose(1,2,0).reshape(nA, nF*subframes)
    for i in range(nA):
        plt.figure()
        plt.plot(t_analogs, data[i])
        plt.xlabel('Time (s)')
        plt.ylabel(labels[i] if i < len(labels) else f'Analog_{i}')
        plt.title(labels[i] if i < len(labels) else f'Analog_{i}')
        plt.tight_layout()
    print(f'Plotted {nA} analog channels.')

plot_all_analogs(analogs, analog_labels, t_analogs)


## Ground Reaction Forces (GRFs)

If your analog labels contain force plate channels (e.g., **Fx, Fy, Fz, Mx, My, Mz**), we will detect them by name and plot.
This is a simple heuristic; exact scaling/orientation depends on your lab configuration.


In [ ]:
# ===== GRF detection & plotting =====
def find_channels(labels, needle):
    idx = []
    for i,l in enumerate(labels):
        if needle.lower() in str(l).lower():
            idx.append(i)
    return idx

def plot_grfs(analogs, labels, t_analogs):
    subf, nA, nF = analogs.shape
    data = analogs.transpose(1,2,0).reshape(nA, nF*subf)
    Fx_i = find_channels(labels, 'Fx')
    Fy_i = find_channels(labels, 'Fy')
    Fz_i = find_channels(labels, 'Fz')

    if not (Fx_i or Fy_i or Fz_i):
        print('No GRF-like channels detected by label.')
        return

    def _sum(indices):
        return np.sum([data[i] for i in indices], axis=0) if indices else None

    Fx = _sum(Fx_i)
    Fy = _sum(Fy_i)
    Fz = _sum(Fz_i)

    if Fx is not None:
        plt.figure(); plt.plot(t_analogs, Fx)
        plt.xlabel('Time (s)'); plt.ylabel('Fx (N)'); plt.title('GRF Fx (sum of detected channels)'); plt.tight_layout()
    if Fy is not None:
        plt.figure(); plt.plot(t_analogs, Fy)
        plt.xlabel('Time (s)'); plt.ylabel('Fy (N)'); plt.title('GRF Fy (sum of detected channels)'); plt.tight_layout()
    if Fz is not None:
        plt.figure(); plt.plot(t_analogs, Fz)
        plt.xlabel('Time (s)'); plt.ylabel('Fz (N)'); plt.title('GRF Fz (sum of detected channels)'); plt.tight_layout()

plot_grfs(analogs, analog_labels, t_analogs)


## Plot markers

We expose a simple function to plot selected markers (X, Y, Z vs time). Pass `names=None` to plot all.


In [ ]:
# ===== Plot markers =====
# points: (4, nMarkers, nFrames) where rows are X,Y,Z,Residual
def plot_markers(points, labels, t_points, names=None, limit=None):
    nM = points.shape[1]
    if names is None:
        names = labels
    idxs = [labels.index(n) for n in names if n in labels]
    if limit is not None:
        idxs = idxs[:limit]
    for i in idxs:
        X = points[0, i, :]
        Y = points[1, i, :]
        Z = points[2, i, :]
        for comp, arr in zip(['X','Y','Z'], [X,Y,Z]):
            plt.figure(); plt.plot(t_points, arr)
            plt.xlabel('Time (s)'); plt.ylabel(f'{comp} (mm)')
            plt.title(f'{labels[i]} — {comp}')
            plt.tight_layout()
    print(f'Plotted {len(idxs)} markers (each with X/Y/Z).')

# Example: plot first 5 markers (uncomment below)
# plot_markers(points, marker_labels, t_points, limit=5)


## Generic C3D Parameter Visualizer (from the prompt)

This widget lets you browse any group/parameter, with helpers for force-plate CORNERS and CHANNEL.


In [ ]:
# ===== Generic C3D Parameter Visualizer =====
# Works alongside your existing widgets
import numpy as np, pandas as pd, ipywidgets as W, matplotlib.pyplot as plt
from IPython.display import display, HTML
from pathlib import Path
import json, html as html_mod

P = c3d['parameters']

def _get_groups():
    return sorted([g for g in P.keys() if isinstance(P[g], dict)])

def _get_params(g):
    if g not in P: return []
    return sorted([k for k,v in P[g].items() if isinstance(v, dict)])

def _val(g,k, default=None):
    try: return P[g][k]['value']
    except Exception: return default

def _to_nd(val):
    # convert nested lists to numpy if possible
    try:
        arr = np.array(val)
        # try to cast numerics
        if arr.dtype == object:
            try:
                arr = arr.astype(float)
            except Exception:
                pass
        return arr
    except Exception:
        return np.array(val, dtype=object)

def _is_numeric(arr):
    return np.issubdtype(arr.dtype, np.number)

def _export_csv(g, p, arr, out_dir='/content'):
    out = Path(out_dir) / f'c3d_param_{g}_{p}.csv'
    try:
        if arr.ndim == 0:
            pd.DataFrame({'value':[arr.item()]}).to_csv(out, index=False)
        elif arr.ndim == 1:
            pd.DataFrame({'index':np.arange(arr.shape[0]), 'value':arr}).to_csv(out, index=False)
        elif arr.ndim == 2:
            pd.DataFrame(arr).to_csv(out, index=False)
        else:
            # flatten higher dims
            flat = arr.reshape(arr.shape[0], -1) if arr.size else arr
            pd.DataFrame(flat).to_csv(out, index=False)
        return str(out)
    except Exception:
        # fallback for non numeric or ragged
        v = _val(g,p)
        out.write_text(json.dumps(v, ensure_ascii=False, indent=2), encoding='utf-8')
        return str(out)

# special plots for force plates
def _plot_fp_corners(arr):
    # expect shape (plates, 3, 4) or (3,4) or flat 12
    A = np.array(arr)
    if A.ndim == 1 and A.size == 12:
        A = A.reshape(1,3,4)
    if A.ndim == 2 and A.shape == (3,4):
        A = A.reshape(1,3,4)
    if A.ndim != 3 or A.shape[1] != 3 or A.shape[2] != 4:
        print('Cannot interpret CORNERS shape', A.shape); return
    plt.figure(figsize=(5,5))
    for i in range(A.shape[0]):
        X, Y = A[i,0,:], A[i,1,:]
        Xc = np.r_[X, X[0]]; Yc = np.r_[Y, Y[0]]
        plt.plot(Xc, Yc, marker='o', label=f'Plate {i+1}')
        plt.text(X.mean(), Y.mean(), f'{i+1}')
    plt.gca().set_aspect('equal', adjustable='box')
    plt.xlabel('X m'); plt.ylabel('Y m'); plt.title('FORCE_PLATFORM CORNERS')
    plt.grid(True, alpha=0.3); plt.legend(); plt.tight_layout(); plt.show()

def _plot_fp_channel(arr):
    A = np.array(arr)
    if A.ndim == 1 and A.size % 6 == 0:
        A = A.reshape(6, -1)
    if A.ndim != 2 or A.shape[0] != 6:
        print('Expected shape 6 x plates, got', A.shape); return
    df = pd.DataFrame(A, index=['Fx','Fy','Fz','Mx','My','Mz'])
    display(df)

# widgets
g_dd  = W.Dropdown(options=_get_groups(), description='Group:', layout=W.Layout(width='280px'))
p_dd  = W.Dropdown(options=_get_params(_get_groups()[0]) if _get_groups() else [], description='Param:', layout=W.Layout(width='320px'))
view_dd = W.Dropdown(options=['Auto', 'Line', 'Heatmap', 'Scatter 2D', 'Table'], value='Auto', description='View:', layout=W.Layout(width='200px'))
dim_dd  = W.Dropdown(options=['auto','rows','cols'], value='auto', description='Plot along:', layout=W.Layout(width='180px'))
export_btn = W.Button(description='Export CSV', icon='download')
out = W.Output()

def on_group_change(change):
    p_dd.options = _get_params(g_dd.value)
    if p_dd.options:
        p_dd.value = p_dd.options[0]
    refresh()

def refresh(*_):
    out.clear_output(wait=True)
    g = g_dd.value; p = p_dd.value
    val = _val(g,p)
    with out:
        if val is None:
            print('No value')
            return
        arr = _to_nd(val)
        print(f"{g}  {p}  shape {getattr(arr,'shape',())}  dtype {getattr(arr,'dtype','')}")
        # force plate helpers
        if g == 'FORCE_PLATFORM' and p.upper() == 'CORNERS' and _is_numeric(arr):
            _plot_fp_corners(arr); return
        if g == 'FORCE_PLATFORM' and p.upper() == 'CHANNEL':
            _plot_fp_channel(arr); return

        if not _is_numeric(arr):
            # show as table of strings
            try:
                if isinstance(val, list) and all(not isinstance(x, (list,tuple)) for x in val):
                    df = pd.DataFrame({p: [str(x) for x in val]})
                else:
                    df = pd.DataFrame(val)
                display(df)
            except Exception:
                print(val)
            return

        # numeric plots
        try:
            if arr.ndim == 0:
                print(arr.item())
            elif arr.ndim == 1:
                plt.figure(figsize=(9,3))
                plt.plot(np.arange(arr.size), arr)
                plt.xlabel('Index'); plt.ylabel('Value'); plt.title(f'{g}.{p}')
                plt.grid(True, alpha=0.3); plt.tight_layout(); plt.show()
            elif arr.ndim == 2:
                mode = view_dd.value
                if mode in ['Auto','Heatmap']:
                    plt.figure(figsize=(6,4))
                    plt.imshow(arr, aspect='auto', origin='lower')
                    plt.colorbar(label='Value')
                    plt.title(f'{g}.{p} heatmap')
                    plt.tight_layout(); plt.show()
                elif mode in ['Line']:
                    # plot each row as a line
                    plt.figure(figsize=(9,4))
                    for i in range(arr.shape[0]):
                        plt.plot(arr[i], alpha=0.6)
                    plt.title(f'{g}.{p} line by row')
                    plt.xlabel('Index'); plt.ylabel('Value')
                    plt.grid(True, alpha=0.3); plt.tight_layout(); plt.show()
                elif mode in ['Scatter 2D']:
                    # take first two cols if present
                    x = arr[:,0] if arr.shape[1] >= 1 else np.arange(arr.shape[0])
                    y = arr[:,1] if arr.shape[1] >= 2 else np.zeros(arr.shape[0])
                    plt.figure(figsize=(5,5))
                    plt.scatter(x,y, s=12)
                    plt.title(f'{g}.{p} scatter')
                    plt.xlabel('col0'); plt.ylabel('col1')
                    plt.grid(True, alpha=0.3); plt.tight_layout(); plt.show()
                else:
                    display(pd.DataFrame(arr))
            else:
                # higher dims fallback
                flat = arr.reshape(arr.shape[0], -1)
                if view_dd.value in ['Auto','Heatmap']:
                    plt.figure(figsize=(6,4))
                    plt.imshow(flat, aspect='auto', origin='lower')
                    plt.colorbar(label='Value')
                    plt.title(f'{g}.{p} flattened heatmap')
                    plt.tight_layout(); plt.show()
                else:
                    display(pd.DataFrame(flat))
        except Exception as e:
            print('Plot error:', e)
            try:
                display(pd.DataFrame(arr))
            except Exception:
                print(arr)

def on_export(_):
    g = g_dd.value; p = p_dd.value
    path = _export_csv(g, p, _to_nd(_val(g,p)))
    display(HTML(f"Saved <a href='sandbox:{path}' target='_blank'>{html_mod.escape(Path(path).name)}</a>"))

g_dd.observe(on_group_change, names='value')
p_dd.observe(lambda _: refresh(), names='value')
view_dd.observe(lambda _: refresh(), names='value')
export_btn.on_click(on_export)

ui = W.VBox([
    W.HBox([g_dd, p_dd]),
    W.HBox([view_dd, dim_dd, export_btn]),
    out
])
display(ui)
refresh()


## One-click: Markers & GRFs

Run the cell below to quickly preview first few markers (X/Y/Z) and detected GRFs.


In [ ]:
# ===== Quick preview =====
# Plot first 3 markers
plot_markers(points, marker_labels, t_points, limit=3)
# Plot detected GRFs
plot_grfs(analogs, analog_labels, t_analogs)
